In [1]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy.sparse import hstack
import joblib

In [2]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def clean_essay(text, remove_stopwords=True, lemmatize=True):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^a-z\s]', '', text)
    if remove_stopwords:
        text = ' '.join([w for w in text.split() if w not in ENGLISH_STOP_WORDS])
    if lemmatize:
        text = ' '.join([lemmatizer.lemmatize(w) for w in text.split()])
    return text

new_essay = '''Technology has revolutionized nearly every aspect of modern life, and education is no exception. Over the past few decades, advancements in digital tools, the internet, and communication technologies have dramatically changed how students learn and how teachers teach. Traditional classrooms have been supplemented—or in some cases replaced—by online learning platforms, interactive software, and digital resources. This transformation has expanded access to education, making it possible for learners from all over the world to study virtually anything at any time.
One of the most significant benefits of technology in education is the ability to access a vast amount of information quickly. Students no longer rely solely on textbooks; instead, they can explore academic articles, videos, simulations, and online lectures. This access encourages independent learning, fosters curiosity, and allows students to explore topics in greater depth than ever before. Additionally, digital tools often provide interactive features that enhance understanding, such as quizzes, animations, and real-time feedback.
Technology also enables personalized learning. Adaptive software can analyze students’ strengths and weaknesses and tailor lessons accordingly. For instance, a student struggling with mathematics can receive additional practice problems or explanatory videos, while a student excelling in the subject can move on to more advanced concepts. This individualized approach helps ensure that every learner progresses at their own pace, reducing frustration and enhancing motivation.
Collaboration is another area where technology has had a profound impact. Online platforms allow students to work together on projects regardless of geographic boundaries. Tools like shared documents, discussion boards, and video conferencing facilitate communication and teamwork, skills that are increasingly important in the globalized workforce. Teachers can also monitor group activities, provide guidance, and assess contributions efficiently
Despite these advantages, technology in education comes with challenges. Overreliance on digital tools can lead to reduced critical thinking skills if students merely consume information without analyzing it. Additionally, there is a digital divide: not all students have equal access to devices or high-speed internet, which can exacerbate educational inequalities. Teachers must also adapt their methods and continuously learn new tools, which can be time-consuming and stressful
In conclusion, technology has fundamentally transformed modern education, offering unprecedented opportunities for learning, personalization, and collaboration. While challenges exist, careful integration of digital tools can enhance educational outcomes and prepare students for a rapidly evolving world. By combining traditional teaching methods with innovative technologies, educators can create an engaging and inclusive learning environment that equips students with the knowledge, skills, and adaptability required for the future.'''

new_essay_clean = clean_essay(new_essay)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Faiz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
import joblib

# Load model & vectorizer
model_forest = joblib.load("E:/Smart_Essay_Scorer/models/Random_forest_model.pkl")
tfidf = joblib.load("E:/Smart_essay_Scorer/models/tfidf_vectorizer.pkl")
model_xg = joblib.load("E:/Smart_Essay_Scorer/models/model_xg.pkl")
model_lgb = joblib.load("E:/Smart_Essay_Scorer/models/model_lgb.pkl")


In [4]:
# Numeric features
word_count = len(new_essay_clean.split())
sent_count = new_essay_clean.count('.') + 1
words_per_sentence = word_count / max(sent_count,1)
char_count = len(new_essay_clean)
avg_word_length = char_count / max(word_count,1)

X_numeric_new = np.array([[word_count, sent_count, word_count, words_per_sentence, char_count, avg_word_length]])
X_text_new = tfidf.transform([new_essay_clean])
X_new_final = hstack([X_numeric_new, X_text_new])

# Predict
predicted_score = model_xg.predict(X_new_final)
print("Predicted Score:", predicted_score[0])
predicted_score_rounded = round(predicted_score[0])
print("Rounded Predicted Score:", predicted_score_rounded)

Predicted Score: 2.6980214
Rounded Predicted Score: 3
